In [1]:
%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
import seaborn as sns
from os.path import join
from collections import defaultdict
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
import dvu
import sys
import warnings
sys.path.append('../notebooks')
from tqdm import tqdm
from sasc.config import FMRI_DIR, STORIES_DIR, RESULTS_DIR
from neuro.config import repo_dir, PROCESSED_DIR
from neuro import analyze_helper, viz
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples
# flatmaps_per_question = __import__('06_flatmaps_per_question')
import viz
import gemv

Note, this notebook requires first running `03_export_qa_flatmaps.ipynb` into `df_qa_dict.pkl` files for each subject.

### load gemv average flatmaps

In [2]:
gemv_flatmaps_pilot = joblib.load(join(
    RESULTS_DIR, "processed", "flatmaps", 'resps_avg_dict_pilot.pkl'))
gemv_flatmaps_pilot5 = joblib.load(join(
    RESULTS_DIR, "processed", "flatmaps", 'resps_avg_dict_pilot5.pkl'))
gemv_flatmaps_dict = gemv_flatmaps_pilot | gemv_flatmaps_pilot5
qa_questions_list, gemv_questions_list = gemv.get_matched_lists()

In [3]:
# # check that gpt4 was run for all the questions
# subject = 'UTS02'
# setting = 'individual_gpt4'
# flatmaps_qa = joblib.load(
#     join(PROCESSED_DIR, subject.replace('UT', ''), setting + '.pkl'))
# for k in qa_list:
#     assert k in flatmaps_qa.keys()

### make comparisons

In [ ]:
# settings = ['full_35']
# settings = ['full_35_wordrate']
# settings = ['full_35_pc']
# settings = ['full_neurosynth']
# settings = ['full_35_gpt4_pc']
settings = ['individual_gpt4_pc']
# settings = ['shapley_35_gpt4_pc']
# settings = ['shapley_35', 'individual_gpt4_pc', 'shapley_35_gpt4_pc']
# settings = ['individual_gpt4_ndel=1_pc']
# settings = ['full_35_gpt4_ndel=1_pc']
# settings = ['full_35_ndel=8_pc']
# settings = ['full_35', 'full_35_wordrate']
# settings = ['individual_gpt4', 'individual_gpt4_wordrate',]
# settings = ['individual_gpt4', 'individual_gpt4_wordrate', 'shapley_35']
# settings = ['shapley_35']
# settings = ['individual_35']
# settings = ['individual_gpt4']
# , 'individual_35', full_35', 'shapley_35', 'individual_gpt4',
# for setting in ['individual_gpt4', 'shapley_35']:
# for setting in ['individual_gpt4', 'individual_gpt4_wordrate']:
# , 'shapley_35']:
# for setting in ['individual_gpt4']:
# for setting in ['individual_gpt4', 'individual_gpt4_wordrate']:
# , 'individual_gpt4', 'individual_gpt4_wordrate']:

# corrs_mask_per_question = False
corrs_mask_per_question = False

In [ ]:
# load qa weights
subject = 'UTS02'
corrs_df_dict = {}
frac_voxels_to_keep_list = [0.001, 0.01, 0.05, 0.1, 0.25, 0.5, 1]

# corrs used for masking
corrs_test = joblib.load(join(PROCESSED_DIR, subject.replace(
    'UT', ''), 'corrs_test_35.pkl')).values[0]
corrs_test_individual_dict = joblib.load(join(PROCESSED_DIR, subject.replace(
    'UT', ''), 'corrs_test_individual_gpt4_qs_35.pkl'))

for j, frac_voxels_to_keep in enumerate(frac_voxels_to_keep_list):
    flatmaps_qa_list = defaultdict(list)
    for setting in settings:
        flatmaps_qa_dict = joblib.load(
            join(PROCESSED_DIR, subject.replace('UT', ''), setting + '.pkl'))
        for q in flatmaps_qa_dict.keys():
            flatmaps_qa_list[q].append(flatmaps_qa_dict[q])
    # print('lens', [len(flatmaps_qa_list[q]) for q in flatmaps_qa_list.keys()])
    flatmaps_qa_dict = {
        q: np.mean(flatmaps_qa_list[q], axis=0)
        # q: np.max(flatmaps_qa_list[q], axis=0)
        # q: np.median(flatmaps_qa_list[q], axis=0)
        for q in flatmaps_qa_list.keys()}

    # select what to plot
    # df_qa_dict = joblib.load(f'df_qa_dict_{subject.replace("UT", "")}.pkl')
    questions_names_df = pd.DataFrame({
        'qa': qa_questions_list,
        'gemv': gemv_questions_list,
    })
    # filter only questions that exist in the flatmaps
    questions_names_df = questions_names_df[questions_names_df['qa'].isin(
        flatmaps_qa_dict.keys())]

    if isinstance(flatmaps_qa_dict, pd.DataFrame):
        questions_names_df = questions_names_df[questions_names_df['qa'].isin(
            flatmaps_qa_dict.index)]
        flatmaps_qa_list = flatmaps_qa_dict.loc[questions_names_df['qa'].values]['weights'].values
    else:
        flatmaps_qa_list = [flatmaps_qa_dict[q]
                            for q in questions_names_df['qa'].values]
    flatmaps_gemv_list = [
        gemv_flatmaps_dict[q]
        for q in questions_names_df['gemv'].values]
    titles_gt = questions_names_df['gemv'].apply(
        lambda x: x[0]).astype(str)

    # mask flatmaps by corr
    # corrs_test = joblib.load(join(PROCESSED_DIR, subject.replace(
    #     'UT', ''), 'corrs_test_35.pkl')).values[0]
    # corrs_test_mask = (corrs_test >= np.percentile(
    #     corrs_test, 100 * (1 - frac_voxels_to_keep))).astype(bool)

    # flatmaps_qa = [flatmaps_qa[i][corrs_test_mask]
    #                for i in range(len(flatmaps_qa))]
    # flatmaps_gemv = [flatmaps_gemv[i][corrs_test_mask]
    #                  for i in range(len(flatmaps_gemv))]

    # mask flatmaps by extreme vals
    for i in range(len(flatmaps_qa_list)):
        if frac_voxels_to_keep < 1:
            # mask based on extreme values
            # mask = np.abs(flatmaps_qa_list[i]) >= np.percentile(
            #     np.abs(flatmaps_qa_list[i]), 100 * (1 - frac_voxels_to_keep))

            # mask based on corrs
            if corrs_mask_per_question:
                q = questions_names_df['qa'].values[i]
                mask = (corrs_test_individual_dict[q] > np.percentile(
                    corrs_test_individual_dict[q], 100 * (1 - frac_voxels_to_keep))).astype(bool)
            else:
                mask = (corrs_test > np.percentile(
                    corrs_test, 100 * (1 - frac_voxels_to_keep))).astype(bool)
        else:
            mask = np.ones_like(flatmaps_qa_list[i]).astype(bool)

        flatmaps_qa_list[i] = flatmaps_qa_list[i][mask]
        flatmaps_gemv_list[i] = flatmaps_gemv_list[i][mask]

    # # save flatmaps
    # if frac_voxels_to_keep in [0.1, 1]:

    #     # apply mask as nans
    #     for i in range(len(flatmaps_qa)):
    #         flatmaps_qa[i][~corrs_test_mask] = np.nan
    #         flatmaps_gemv[i][~corrs_test_mask] = np.nan

    #     for i in tqdm(range(len(flatmaps_qa))):
    #         sasc.viz.quickshow(
    #             flatmaps_qa[i],
    #             subject=subject,
    #             fname_save=join(repo_dir, 'qa_results', 'gemv', subject,
    #                             f'frac_voxels={frac_voxels_to_keep}',
    #                             setting, f'{titles_gt[i]}.png'),
    #             cmap='RdYlBu_r',
    #         )

    #         sasc.viz.quickshow(
    #             flatmaps_gemv[i],
    #             subject=subject,
    #             fname_save=join(repo_dir, 'qa_results', 'gemv', subject,
    #                             f'frac_voxels={frac_voxels_to_keep}',
    #                             'gemv', f'{titles_gt[i]}.png'),
    #             cmap='RdYlBu_r',
    #         )

    # shuffle gemv
    # random.shuffle(flatmaps_gemv)

    # print num nas
    # print('nas', [np.isnan(flatmaps_gemv[i]).sum()
    #   for i in range(len(flatmaps_gemv))])

    corrs_mat = viz._calc_corrs(
        flatmaps_qa_list,
        flatmaps_gemv_list,
        titles_qa=[analyze_helper.abbrev_question(q)
                   for q in questions_names_df['qa'].astype(str)],
        titles_gt=titles_gt,
    )
    # corrs_df = pd.DataFrame({'corrs': np.diag(
    # corrs.values), 'questions': corrs.columns}).sort_values('corrs', ascending=False)
    # corrs_df_dict[frac_voxels_to_keep] = corrs_df.copy()
    corrs_df_dict['questions'] = corrs_mat.columns
    corrs_df_dict[f'corrs_{frac_voxels_to_keep}'] = np.diag(corrs_mat.values)


# actually make plot
corrs_df = pd.DataFrame(corrs_df_dict)
corrs_df = corrs_df.sort_values('corrs_1', ascending=False)
# corrs_df.to_pickle(join(repo_dir, 'qa_results', 'gemv',
# setting + '_corrs_df.pkl'))
for j, frac_voxels_to_keep in enumerate(frac_voxels_to_keep_list):
    colors = sns.color_palette('viridis', len(frac_voxels_to_keep_list))
    viz.corr_bars(
        corrs_df['corrs_' + str(frac_voxels_to_keep)],
        questions=corrs_df['questions'],
        out_dir_save=join(repo_dir, 'qa_results', 'gemv',
                          'corrs_' + setting),
        xlab='GEM-V flatmap <> QA',
        color=colors[j],
        label=f'{frac_voxels_to_keep * 100:.2f}%',
    )
    # plt.savefig(join(out_dir_save, 'corrs_barplot.pdf'), bbox_inches='tight')
    # plt.savefig(join(out_dir_save, 'corrs_barplot.png'),
    #             bbox_inches='tight', dpi=300)
    # plt.show()

plt.title('___'.join(settings))
# center legend text
plt.legend(title='Fraction of\nbest-predicted voxels',
           labelcolor='linecolor')
# plt.tight_layout()
# plt.savefig(join(repo_dir, 'qa_results', 'gemv', subject, 'corrs_barplot.pdf'),
# bbox_inches='tight')
out_fname = 'corrs_' + '___'.join(settings)
if corrs_mask_per_question:
    out_fname += '_mask_per_question'
plt.savefig(join(repo_dir, 'qa_results', 'gemv', subject, out_fname + '.pdf'),
            bbox_inches='tight', dpi=300)
joblib.dump(corrs_df, join(repo_dir, 'qa_results', 'gemv',
                           subject, out_fname + '.pkl'))
plt.show()

In [ ]:
# aggregate results
corrs_results_dir = join(repo_dir, 'qa_results', 'gemv', subject)
d = defaultdict(list)
for fname in os.listdir(corrs_results_dir):
    if fname.startswith('corrs_') and fname.endswith('.pkl'):
        d['setting'].append(fname.replace('corrs_', '').replace('.pkl', ''))
        x = joblib.load(join(corrs_results_dir, fname))
        d['corrs_df'].append(joblib.load(join(corrs_results_dir, fname)))
        if not len(x) == len(gemv_questions_list):
            warnings.warn(
                'not all questions were computed for ' + str(d['setting'][-1]))
# assert all([x.shape[0] == d['corrs_df'][0].shape[0] for x in d['corrs_df']])
avg_df = pd.DataFrame([x.drop(columns='questions').mean(axis=0)
                      for x in d['corrs_df']], index=d['setting'])
avg_df = avg_df.sort_values(by='corrs_1', ascending=False)
avg_df.style.background_gradient(cmap='viridis').format("{:.3f}")

### compute p-values with eng1000

In [ ]:
for frac_voxels_to_keep in tqdm(frac_voxels_to_keep_list):
    eng1000_dir = join(PROCESSED_DIR, subject.replace(
        'UT', ''), 'eng1000_weights.pkl')
    pvals = viz.compute_pvals(flatmaps_qa_list, frac_voxels_to_keep,
                              corrs_df[f'corrs_{frac_voxels_to_keep}'].values, eng1000_dir=eng1000_dir, mask_corrs=corrs_test)

    # get what fraction of 'corrs_perm_eng1000' column is greater than 'corrs'
    corrs_df[f'pval_{frac_voxels_to_keep}'] = pvals

In [ ]:
# add average row
corrs_viz = corrs_df.set_index('questions').sort_values(
    by='pval_0.001')
# by='corrs_1')
corrs_viz.loc['AVG'] = corrs_viz.mean()
corrs_viz.loc['AVG_FIRST10'] = corrs_viz.head(10).mean()

# format scientific notation
print('___'.join(settings))
corrs_viz.style.background_gradient().format(precision=3)

In [ ]:
from ridge_utils.SemanticModel import SemanticModel
from neuro.config import em_data_dir
eng1000 = SemanticModel.load(join(em_data_dir, "english1000sm.hf5"))

In [ ]:
word_to_index_dict = eng1000.vindex

index_to_wordlist_dict = defaultdict(list)
for k, v in word_to_index_dict.items():
    index_to_wordlist_dict[v].append(k.decode())
# check if all lists are length 1
assert all([len(v) == 1 for v in index_to_wordlist_dict.values()])

In [ ]:
words_list_985 = list(word_to_index_dict.keys())
dog_vec = eng1000['dog'.encode()]
words_list_985[np.argmax(dog_vec) + 1]

In [ ]:
for k in corrs_viz.index.values[:-2]:
    first_word = k.split()[0]
    if first_word.endswith('s'):
        first_word = first_word[:-1]
    rename = {
        'measurement': 'measure',
        'dialogue': 'talk',
        'sensory': 'sense',
        'negation': 'negative',
        'cultural': 'culture',
    }
    first_word = rename.get(first_word, first_word)

    print(k, '--', first_word,
          eng1000.find_words_like_word(first_word.encode(), n=100))

# Look at merged flatmaps

In [ ]:
corrs_df = pd.read_pickle(join(repo_dir, 'qa_results', 'gemv',
                               setting + '_corrs_df.pkl'))

In [ ]:
img_parent_dir = join(repo_dir, 'qa_results', 'gemv',
                      subject, 'frac_voxels=0.1')
img_dir1 = join(img_parent_dir, 'gemv')
img_dir2 = join(img_parent_dir, setting)

# read images and combine them with their filenames on a single plot
# fnames = os.listdir(img_dir1)
# fnames = [f for f in fnames if f.endswith('.png')]
# only keep the ones that are in both directories
# fnames = [f for f in fnames if f in os.listdir(img_dir2)]


# corrs = corrs.sort_values('corrs', ascending=False)
fnames = [v + '.png' for v in corrs_df['questions'].values]

n = len(fnames)
C = 4
R = int(np.ceil(n / C))

fig, axs = plt.subplots(R, C, figsize=(C * 3.2, R * 1))
axs = axs.flatten()
for i in range(len(axs)):
    axs[i].axis('off')
for i, fname in enumerate(fnames):
    img1 = plt.imread(join(img_dir1, fname))
    img2 = plt.imread(join(img_dir2, fname))
    axs[i].imshow(np.concatenate([img1, img2], axis=1))
    axs[i].set_title(
        f'{fname[:-4]} ({corrs_df["corrs"].values[i]:0.3f})', fontsize=8)

# add text in bottom right of figure
fig.text(0.99, 0.01, f'{subject}\nGEMV on left, QA on right',
         ha='right', va='bottom', fontsize=8)
plt.tight_layout()
plt.savefig(join(img_parent_dir, f'flatmaps_{setting}_{subject}.png'), dpi=300)
plt.close()